In [ ]:
import os 
import pandas as pd
import numpy as np
import torch
from torch import nn
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'You are using {device}')

You are using cuda


### Visual Style 
Bu çalışmada yapay zeka modellerinin özellik çıkarımından yararlanarak sanatsal bir tablo günlük bir resime entegre edilmiştir. Bunun için bir CNN modeli olan VGG19 modelinin belli başlı katmanları seçilerek günlük görüntü ve sanatsal görüntü üzerinden özellik çıkarımı yapılmış, Gram matrisi aracılığıyla kayıp fonksiyonu hesaplanmış ve optimizer fonksiyonuna görüntü girdi olarak verilerek bütün bu işlemlerin görüntü üzerinden olması sağlanmıştır.

In [ ]:
class VisualModel(nn.Module):
    def __init__(self) -> None:
        super(VisualModel, self).__init__()
        self.selected_layers = [0, 5, 10, 19, 28] #Makalede seçilen katman numaraları
        self.model = models.vgg19(weights=models.VGG19_Weights.DEFAULT)
        self.feats = self.model.features[:29]
    def forward(self, x):
        features = []
        for layer_num, layer in enumerate(self.feats):
            x = layer(x) #Gelen görüntü her katmandan geçirilip sadece belli başlı katmandaki özellikleri seçilecektir. Bütün katmanlardan geçirmeyip sadece seçilecek katmanları eklersek istenilen sonucu alamadık.
            if layer_num in self.selected_layers:
                features.append(x)
        return features

In [20]:
test_model = VisualModel().to(device)
test_model

VisualModel(
  (model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (1

In [ ]:
img_size = 200
transform = transforms.Compose([
    transforms.Resize(size = (img_size, img_size)),
    transforms.ToTensor()
    #Normalize işlemi yapmadık normalizeli haliyle resimleri tersine çevirmemiz gerekirdi buna gerek yok.
    #transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
    ])

In [22]:
def load_image(img_path, transform = None):
  img = Image.open(img_path).convert("RGB")
  if transform:
    img = transform(img).unsqueeze(0)
  return img.to(device)

In [23]:
original_image = load_image("/kaggle/input/vst-dataset/1.png", transform = transform)
generated_image = load_image("/kaggle/input/vst-dataset/1.png", transform = transform).clone().requires_grad_(True)
style_image = load_image("/kaggle/input/vst-dataset/starry_night.jpg", transform = transform)

In [24]:
model = VisualModel().to(device).eval()

In [20]:
from torchvision.transforms import ToPILImage

# Optimizasyon döngüsü
to_pil_image = ToPILImage()  # Tensörü görüntüye çevirmek için

num_steps = 10000
learning_rate = 0.001
alpha = 1
beta = 0.01
optimizer = torch.optim.Adam([generated_image], lr = learning_rate)
for step in range(num_steps):
  #Orijinal resim, üretilecek resim ve stil resmi modelden geçirilerek feature çıkarılması sağlanır.
  generated_features = model(generated_image)
  original_features = model(original_image)
  style_features = model(style_image)
  #Tabi bunların her birinin kaybı da olacaktır.
  style_loss = 0
  original_loss = 0
  generated_loss = 0
  for gen_feat, og_feat, style_feat in zip(generated_features, original_features, style_features):
    batch_size, channel, height, width = gen_feat.shape
    #Makalede de bahsedildiği üzere orijinal içerik kaybı MSE dir Bu oluşturulan resim kaybına eklenir.
    original_loss = torch.mean((gen_feat - og_feat) ** 2)
    generated_loss += original_loss
    #Gram matrisi
    G = torch.mm(gen_feat.view(channel, height * width), gen_feat.view(channel, height * width).t())
    A = torch.mm(style_feat.view(channel, height * width), style_feat.view(channel, height * width).t())
    style_loss += torch.mean((G - A) ** 2)
  total_loss = alpha * style_loss + beta * generated_loss
  optimizer.zero_grad()
  total_loss.backward()
  optimizer.step()

  if step % 200 == 0:
      print(f"Step [{step}/{num_steps}], Total Loss: {total_loss.item():.4f}")
      img_to_save = to_pil_image(generated_image.squeeze(0).cpu())
      img_to_save.save(f"generated_image_step_{step}.png")


Step [0/10000], Total Loss: 13814236.0000
Step [200/10000], Total Loss: 691643.6875
Step [400/10000], Total Loss: 332796.0938
Step [600/10000], Total Loss: 215951.3438
Step [800/10000], Total Loss: 148867.4219
Step [1000/10000], Total Loss: 108951.5547
Step [1200/10000], Total Loss: 87261.8047
Step [1400/10000], Total Loss: 75337.5703
Step [1600/10000], Total Loss: 67608.5000
Step [1800/10000], Total Loss: 61964.7305
Step [2000/10000], Total Loss: 57452.6211
Step [2200/10000], Total Loss: 53689.2773
Step [2400/10000], Total Loss: 50507.9648
Step [2600/10000], Total Loss: 47692.2539
Step [2800/10000], Total Loss: 45229.3711
Step [3000/10000], Total Loss: 43053.9180
Step [3200/10000], Total Loss: 41145.9453
Step [3400/10000], Total Loss: 39472.9375
Step [3600/10000], Total Loss: 37928.5430
Step [3800/10000], Total Loss: 36544.1445
Step [4000/10000], Total Loss: 35282.7305
Step [4200/10000], Total Loss: 34118.4648
Step [4400/10000], Total Loss: 33043.0586
Step [4600/10000], Total Loss: 32